In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

## 6.23

In [2]:
df = pd.read_csv("./data/T11-5.DAT", sep="\s+", header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "iris_class"])
df_width = df[["sepal_width", "petal_width", "iris_class"]]

In [3]:
df_width_group = df_width.groupby("iris_class")

In [4]:
x_bar = df_width[["sepal_width", "petal_width"]].mean()
x_l_bar = df_width_group.mean()
n_l = df_width_group.count()

In [5]:
B = (n_l.values.T * (x_l_bar - x_bar).T).dot(x_l_bar - x_bar)

In [6]:
W = np.zeros((2, 2))
for i in range(3):
    W += df_width_group.get_group(i + 1)[["sepal_width", "petal_width"]].cov(ddof=0) * n_l.values[i, 0]

In [7]:
print(f"B:{B}")
print(f"W:{W}")
print(f"B+W:{B+W}")

B:             sepal_width  petal_width
sepal_width    11.344933   -22.932667
petal_width   -22.932667    80.413333
W:             sepal_width  petal_width
sepal_width      16.9620       4.8084
petal_width       4.8084       6.1566
B+W:             sepal_width  petal_width
sepal_width    28.306933   -18.124267
petal_width   -18.124267    86.569933


In [8]:
for i in range(2):
    temp = stats.t.ppf(1 - 0.05/(2 * 3 * (3 - 1)), df=150-3) * np.sqrt(W.values[i,i] / (150-3) * (1/50+1/50))
    print(f"x_3{i+1}-x_1{i+1}: ({x_l_bar.values[2, i] - x_l_bar.values[0, i] - temp}, {x_l_bar.values[2, i] - x_l_bar.values[0, i] + temp})")
    print(f"x_2{i+1}-x_1{i+1}: ({x_l_bar.values[1, i] - x_l_bar.values[0, i] - temp}, {x_l_bar.values[1, i] - x_l_bar.values[0, i] + temp})")
    print(f"x_3{i+1}-x_2{i+1}: ({x_l_bar.values[2, i] - x_l_bar.values[1, i] - temp}, {x_l_bar.values[2, i] - x_l_bar.values[1, i] + temp})")

x_31-x_11: (-0.6356943622645526, -0.2723056377354478)
x_21-x_11: (-0.8396943622645523, -0.47630563773544754)
x_31-x_21: (0.02230563773544733, 0.3856943622645521)
x_32-x_12: (1.6705354763538884, 1.8894645236461112)
x_22-x_12: (0.9705354763538886, 1.1894645236461112)
x_32-x_22: (0.5905354763538887, 0.8094645236461112)


In [9]:
s_j = []
for i in range(3):
    s_j.append(df_width_group.get_group(i + 1)[["sepal_width", "petal_width"]].cov(ddof=1))
s_pool = 1/149 * ((s_j[0] * 49) + (s_j[1] * 49) + (s_j[2] * 49))
l_star = np.linalg.det(s_j[0])**(49/2) * np.linalg.det(s_j[1])**(49/2) * np.linalg.det(s_j[2])**(49/2) / np.linalg.det(s_pool)**(149/2)

In [10]:
l_star

6.7075040297093525e-09

In [11]:
(3/49 - 1/147) * (6+6-1)/(6*3*2)

0.016628873771730914

In [12]:
chi = (1 - (3/49 - 1/147) * (6+6-1)/(6*3*2)) * (-2 * np.log(l_star))

In [13]:
print(stats.chi2.ppf(0.95, df=(2*3*2)/2))
print(chi)

12.591587243743977
37.01416576211768


## 6.31

In [14]:
df = pd.read_csv("./data/T6-17.dat", sep="\s+", header=None, names=["factor1", "factor2", "x1", "x2", "x3"])

In [15]:
df_f1 = df.groupby("factor1")
df_f2 = df.groupby("factor2")
df_f1_f2 = df.groupby(["factor1", "factor2"])

In [16]:
x_bar = df[["x1", "x2", "x3"]].mean()
x_l_bar = df_f1[["x1", "x2", "x3"]].mean()
x_k_bar = df_f2[["x1", "x2", "x3"]].mean()
x_lk_bar = df_f1_f2[["x1", "x2", "x3"]].mean()
g = df["factor1"].nunique()
b = df["factor2"].nunique()
n = 2

In [17]:
ss_f1 = b * n * (x_l_bar - x_bar).T.dot(x_l_bar - x_bar)
ss_f2 = g * n * (x_k_bar - x_bar).T.dot(x_k_bar - x_bar)
ss_interact = n * (x_lk_bar - x_l_bar - x_k_bar + x_bar).T.dot(x_lk_bar - x_l_bar - x_k_bar + x_bar)
df.index = pd.MultiIndex.from_frame(df[["factor1", "factor2"]])
ss_residual = (df[["x1", "x2", "x3"]] - x_lk_bar).T.dot(df[["x1", "x2", "x3"]] - x_lk_bar)

In [18]:
print(ss_f1)
print(ss_f2)
print(ss_interact)
print(ss_residual)

           x1        x2          x3
x1   0.700833  -10.6575    7.129167
x2 -10.657500  162.0675 -108.412500
x3   7.129167 -108.4125   72.520833
          x1         x2          x3
x1  196.1150   365.1825   42.627500
x2  365.1825  1089.0150  414.655000
x3   42.6275   414.6550  284.101667
            x1        x2          x3
x1  205.101667  363.6675  107.785833
x2  363.667500  780.6950  254.220000
x3  107.785833  254.2200   85.951667
         x1       x2       x3
x1  104.205   49.365   76.480
x2   49.365  352.105  121.995
x3   76.480  121.995   94.835


In [19]:
p = 3
l_star = np.linalg.det(ss_residual) / (np.linalg.det(ss_residual + ss_interact))
print(l_star)
chi = -(g * b * (n - 1) - (p+1-(g-1)*(b-1))/2) * np.log(l_star)
print(stats.chi2.ppf(0.95, df=(g-1)*(b-1)*p))
print(chi)

0.07429984180628003
12.591587243743977
12.998232281910447


In [20]:
l_star = np.linalg.det(ss_residual) / (np.linalg.det(ss_f1 + ss_residual))
print(l_star)
chi = -(g * b * (n - 1) - (p+1-(g-1))/2) * np.log(l_star)
print(stats.chi2.ppf(0.95, df=(g-1)*p))
print(chi)

0.10651619790519301
7.814727903251179
10.077561955760036


In [21]:
l_star = np.linalg.det(ss_residual) / (np.linalg.det(ss_f2 + ss_residual))
chi = -(g * b * (n - 1) - (p+1-(b-1))/2) * np.log(l_star)
print(l_star)
print(stats.chi2.ppf(0.95, df=(b-1)*p))
print(chi)

0.012444166211401582
12.591587243743977
21.932516716427894


In [22]:
def two_way_univariate_ANOVA_f(col, df1):
    df1_f1 = df1.groupby("factor1")
    df1_f2 = df1.groupby("factor2")
    df1_f1_f2 = df1.groupby(["factor1", "factor2"])
    x1_bar = df1[col].mean()
    x1_l_bar = df1_f1[col].mean()
    x1_k_bar = df1_f2[col].mean()
    x1_lk_bar = df1_f1_f2[col].mean()
    g1 = df1["factor1"].nunique()
    b1 = df1["factor2"].nunique()
    n1 = 2
    ss_interact1 = n1 * np.sum((x1_lk_bar - x1_l_bar - x1_k_bar + x1_bar)**2)
    df1.index = pd.MultiIndex.from_frame(df1[["factor1", "factor2"]])
    ss_residual1 = np.sum((df1[col] - x1_lk_bar)**2)
    df1 = df1.reset_index(drop=True)
    f_critical = stats.f.ppf(0.95, dfn=(g1-1)*(b1-1), dfd=g1*b1*(n-1))
    return [(ss_interact1 / ((g1-1)*(b1-1))) / (ss_residual1 / (g1*b1*(n-1))), ss_interact1, ss_residual1]

In [23]:
df = pd.read_csv("./data/T6-17.dat", sep="\s+", header=None, names=["factor1", "factor2", "x1", "x2", "x3"])
x1_anova = two_way_univariate_ANOVA_f("x1", df)
df = pd.read_csv("./data/T6-17.dat", sep="\s+", header=None, names=["factor1", "factor2", "x1", "x2", "x3"])
x2_anova = two_way_univariate_ANOVA_f("x2", df)
df = pd.read_csv("./data/T6-17.dat", sep="\s+", header=None, names=["factor1", "factor2", "x1", "x2", "x3"])
x3_anova = two_way_univariate_ANOVA_f("x3", df)

print(x1_anova)
print(x2_anova)
print(x3_anova)
print(stats.f.ppf(0.95, dfn=(g-1)*(b-1), dfd=g*b*(n-1)))

[5.904755050141562, 205.1016666666667, 104.20499999999976]
[6.651666406327646, 780.6949999999981, 352.1049999999998]
[2.718985606579845, 85.95166666666654, 94.83500000000001]
5.143252849784718


In [24]:
df2 = df_f1.get_group(2)
df2_f2 = df2.drop(["factor1", "factor2"], axis=1).groupby("factor2")

In [25]:
df2_f2.mean() - df2_f2.std() / np.sqrt(2) * stats.t.ppf(1 - 0.05/6, df=1)

,x1,x2,x3
factor2,,,
5,7.473664,-221.033826,-161.995949
6,102.769429,81.844813,-63.043647
8,172.108656,36.640392,24.092695


In [26]:
df2_f2.mean() + df2_f2.std() / np.sqrt(2) * stats.t.ppf(1 - 0.05/6, df=1)

,x1,x2,x3
factor2,,,
5,362.626336,481.633826,261.895949
6,297.530571,246.055187,177.543647
8,229.391344,303.959608,108.107305
